Performancetest DB-Scan

In [ ]:
import h5py
import json
import time
import numpy as np
from sklearn.cluster import DBSCAN

#dataset_path = r"C:\Users\roesc\Hochschule\09_Semester\Sensorik\RadarScenes\data"
dataset_path = r"RadarScenes\RadarScenes\data"
db_scan = DBSCAN(eps=4, min_samples=1)  # DBSCAN
performance = 0

for sequence_nr in range(1, 159):  # Alle Sequenzen durchlaufen
    print("Sequence", sequence_nr)    
    sequence_folder = rf"{dataset_path}\sequence_{sequence_nr}"
    hdf_file = rf"{sequence_folder}\radar_data.h5"
    json_file = rf"{sequence_folder}\scenes.json"
    
    with h5py.File(hdf_file, "r") as f:
        radar_dataset = f["radar_data"][:]  
    
    with open(json_file, 'r', encoding='utf-8') as file:
        scenes_json = json.load(file)
    
    scenes = scenes_json["scenes"]
    
    start, end = None, None
    sensor_1, sensor_2, sensor_3, sensor_4 = [], [], [], [] # Wir sollen die Daten sammeln und erst auswerten, wenn wir Daten aus allen vier Sensoren haben

    for class_id in range(12):  # Alle Klassen durchlaufen
        if class_id not in np.unique(radar_dataset["label_id"]):  # Wenn die Klasse in den Daten nicht vorkommt brauchen wir hier gar nicht weitermachen
            continue
    
        for timestamp in scenes:  # Alle Timestamps in der Sequenz durchlaufen
            scene = scenes[timestamp]
            sensor_id = scene["sensor_id"]
            start, end = scene["radar_indices"][0], scene["radar_indices"][1]
            radar_data = radar_dataset[start:end]        
    
            data = radar_data[radar_data["label_id"] == class_id]
            x, y = data["x_cc"], data["y_cc"]

            # Manchmal kommt es hier vor, dass ein leeres Array entsteht, wenn in diesem Zeitraum start:end keine Daten zu der Klasse vorhanden sind. 
            # Das wollen wir natürlich nicht auswerten.
            if len(data) == 0: 
                continue
        
            if sensor_id == 1:
                sensor_1 = [x, y]
            elif sensor_id == 2:
                sensor_2 = [x, y]
            elif sensor_id == 3:
                sensor_3 = [x, y]
            elif sensor_id == 4:
                sensor_4 = [x, y]
        
            if len(sensor_1) > 0 and len(sensor_2) > 0 and len(sensor_3) > 0 and len(sensor_4):  # Erst weiter machen, wenn Daten von allen vier Sensoren da sind
                x_all_sensors = list(sensor_1[0]) + list(sensor_2[0]) + list(sensor_3[0]) + list(sensor_4[0])
                y_all_sensors = list(sensor_1[1]) + list(sensor_2[1]) + list(sensor_3[1]) + list(sensor_4[1])
                
                sensor_1, sensor_2, sensor_3, sensor_4 = [], [], [], []
        
                X = np.array(list(zip(x_all_sensors, y_all_sensors)))
    
                start_time = time.time()
                db_scan_labels = db_scan.fit_predict(X)  # DBSCAN
                end_time = time.time()
    
                performance += end_time - start_time  # Nur die Zeit messen, die von DBSCAN benötigt wird
            
print(f"Performance: {performance}s")

Performancetest Mean-Shift

In [6]:
import h5py
import json
import time
import numpy as np
from sklearn.cluster import MeanShift

dataset_path = r"RadarScenes\RadarScenes\data"
mean_shift = MeanShift(bandwidth=4, bin_seeding=True)  # MeanShift
performance = 0

for sequence_nr in range(1, 159):  # Alle Sequenzen durchlaufen
    print("Sequence", sequence_nr)
    sequence_folder = rf"{dataset_path}\sequence_{sequence_nr}"
    hdf_file = rf"{sequence_folder}\radar_data.h5"
    json_file = rf"{sequence_folder}\scenes.json"
    
    with h5py.File(hdf_file, "r") as f:
        radar_dataset = f["radar_data"][:]
    
    with open(json_file, 'r', encoding='utf-8') as file:
        scenes_json = json.load(file)
    
    scenes = scenes_json["scenes"]
    
    sensor_1, sensor_2, sensor_3, sensor_4 = [], [], [], []  # Daten sammeln, bis alle Sensoren vorhanden sind

    for class_id in range(12):  # Alle Klassen durchlaufen
        if class_id not in np.unique(radar_dataset["label_id"]):
            continue
    
        for timestamp in scenes:
            scene = scenes[timestamp]
            sensor_id = scene["sensor_id"]
            start, end = scene["radar_indices"][0], scene["radar_indices"][1]
            radar_data = radar_dataset[start:end]
    
            data = radar_data[radar_data["label_id"] == class_id]
            x, y = data["x_cc"], data["y_cc"]

            if len(data) == 0:
                continue
        
            if sensor_id == 1:
                sensor_1 = [x, y]
            elif sensor_id == 2:
                sensor_2 = [x, y]
            elif sensor_id == 3:
                sensor_3 = [x, y]
            elif sensor_id == 4:
                sensor_4 = [x, y]
        
            if len(sensor_1) > 0 and len(sensor_2) > 0 and len(sensor_3) > 0 and len(sensor_4):
                x_all_sensors = list(sensor_1[0]) + list(sensor_2[0]) + list(sensor_3[0]) + list(sensor_4[0])
                y_all_sensors = list(sensor_1[1]) + list(sensor_2[1]) + list(sensor_3[1]) + list(sensor_4[1])
                
                sensor_1, sensor_2, sensor_3, sensor_4 = [], [], [], []
        
                X = np.array(list(zip(x_all_sensors, y_all_sensors)))
    
                start_time = time.time()
                mean_shift_labels = mean_shift.fit_predict(X)  # MeanShift
                end_time = time.time()
    
                performance += end_time - start_time
            
print(f"Performance: {performance}s")


Sequence 1
Sequence 2
Sequence 3


c:\Users\linus\miniconda3\Lib\site-packages\sklearn\cluster\_mean_shift.py:293: UserWarning: Binning data failed with provided bin_size=4.000000, using data points as seeds.
  warnings.warn(


Sequence 4
Sequence 5
Sequence 6
Sequence 7
Sequence 8
Sequence 9
Sequence 10
Sequence 11


KeyboardInterrupt: 